## Reloading data and pca data

In [1]:
from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import dask.dataframe as dd


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Dimension reduction and clustering libraries
import umap
import hdbscan
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score

In [2]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
#data = pd.read_csv('Data/customer_data.csv')

In [ ]:
#data_pca = pd.read_csv('Data/pca_customer_data.csv')

In [187]:
data = dd.read_csv('Data/customer_data.csv')

In [188]:
data_pca = dd.read_csv('Data/pca_customer_data.csv')

In [189]:
del data_pca['Unnamed: 0']
del data_pca['UniqueID']
del data['Unnamed: 0']

In [138]:
data_pca.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14
0,-0.363684,-0.071254,0.020346,-0.039878,0.269947,-0.140284,0.768594,0.415667,-0.246301,-0.001240,0.221670,-0.386232,-0.350949,0.762481
1,-0.364945,-0.194907,0.028743,-0.203430,0.119507,0.632649,1.103069,0.025847,-0.007877,-0.107257,-0.131542,-0.144859,-0.015719,-0.018377
2,-0.360729,-0.138560,0.000326,-0.077302,0.066957,-0.059903,0.013442,0.140903,-0.160615,0.094486,0.451730,0.007629,-0.334301,0.782349
3,-0.371532,0.093126,0.702033,-0.209900,-0.375004,0.014823,-0.194265,0.183428,-0.244426,0.095214,0.503775,-0.396859,0.687686,-0.165639
4,-0.356321,-0.147192,0.035836,-0.277386,0.765923,0.116238,-0.367736,-0.052919,0.008150,0.197611,0.270320,-0.330237,-0.316658,0.796324


In [139]:
data.head()

,UniqueID,FrequencyofVisits,Age,Sex,Payer,Facility
0,5962ec84553dcb84e508e2fc45789278,12,37.0,Feminino,ADVANCECARE,CCB
1,547c0d52bbfe1b93b9411d7b55f3bd82,3,37.0,Feminino,ADVANCECARE,HCP
2,1770ad573874abe83cd04bfaf89c8eb6,2,37.0,Feminino,ENTIDADES COM DESCONTO,CCB
3,11b9573f2fc9dd49594cd1f301cb7be6,16,37.0,Feminino,MÉDIS,HCS
4,84fdd6d40ab2a9b95f3c957237225d33,1,37.0,Feminino,PARTICULARES,CCB


## HDBSCAN

In [8]:
from hdbscan import HDBSCAN

In [174]:
#data = data.head(100000)

In [175]:
#data_pca=data_pca.head(100000)

In [190]:
len(data['Age']) # check that the loc worked 

810341

In [191]:
len(data_pca['PC1']) # check that the loc worked 

810341

In [192]:
clusterer_hdbscan = HDBSCAN(min_cluster_size=24000)

In [ ]:
hdbscan_labels = clusterer_hdbscan.fit_predict(data_pca)

In [180]:
np.unique(hdbscan_labels)

array([-1,  0,  1,  2,  3,  4])

In [181]:
cluster_id = hdbscan_labels

In [182]:
data['cluster_id'] = cluster_id

In [183]:
data['cluster_id']= data['cluster_id'].replace(-1,(len(np.unique(hdbscan_labels))-1)) #figure out how to replace the -1 with the length of clusters 

In [184]:
data.cluster_id.unique()

array([5, 3, 0, 2, 4, 1])

In [185]:
del data['UniqueID']

In [169]:
categorical_data = data.select_dtypes([object, "category"])

In [111]:
def cluster_summary(cluster_id):
    cluster = data[data.cluster_id==cluster_id]
    cluster_summary = cluster[categorical_data.columns].mode().to_dict(orient="records")[0]
    cluster_summary.update(cluster.median().to_dict())
    return cluster_summary

In [112]:
def cluster_comparison(*cluster_ids):
    summaries = []
    for cluster_id in cluster_ids:
        summaries.append(cluster_summary(cluster_id))
    return pd.DataFrame(summaries).set_index("cluster_id").T

In [186]:
cluster_comparison(0,1,2,3,4,5)

cluster_id,0.0,1.0,2.0,3.0,4.0,5.0
Age,55,56,54,53,54,55
Facility,HCD,HCD,HCD,CCA,HCV,HCP
FrequencyofVisits,6,4,5,4,4,4
Payer,MÉDIS,MULTICARE,MULTICARE,MULTICARE,ADSE,ADSE
Sex,Feminino,Masculino,Feminino,Feminino,Feminino,Feminino


In [114]:
'''from hdbscan import approximate_predict

clusterer_hdbscan2 = HDBSCAN(prediction_data=True)
hdbscan_labels2 = clusterer_hdbscan.fit_predict(data_pca)'''

## UMAPS 

In [ ]:
clusterable_embedding = umap.UMAP(
    n_neighbors=30,
    min_dist=0.0,
    n_components=2,
    random_state=42,
).fit_transform(data_pca)